In [1]:
import pysal
import numpy
import scipy
import pandas
import shapefile as shp
from sklearn import tree
from pymining import seqmining
import matplotlib.pyplot as plt
from sklearn import preprocessing

%matplotlib inline
plt.style.use('ggplot')
pandas.set_option('display.max_rows', 100, "display.max_columns", 100)

In [2]:
db_business = pysal.open('./dataset/Data/CLT_Business_Attri.dbf','r')
df_business = pandas.DataFrame(db_business[:], columns=db_business.header)

db_crimes = pysal.open('./dataset/Data/CrimeIncident_CMPD_2010.dbf','r')
df = pandas.DataFrame(db_crimes[:], columns=db_crimes.header)
df.shape

(67595, 53)

In [4]:
street2block = df_business.set_index('ST_NAME')["Block_id"].to_dict()
df["BLOCKID10"] = df["Street_Nam"].map(lambda street: street2block.get(street, "not found"))

df = df[df["BLOCKID10"] != "not found"]
df.shape

(50455, 54)

In [5]:
date_dict = dict()
for i, row in df.iterrows():
    date = row["Report_Dat"][:-5]
    t = (row["Report_Dat"][-4:], row["NIBRSclass"], row["BLOCKID10"])
    date_dict[date] = date_dict.get(date, []) + [t]

In [13]:
for i in date_dict:
    g = [k for k in date_dict[i] if k[2][-4] == "3"]
    print g
    print "\n"

[(u'0023', u'Simple Assault', u'371190019103000'), (u'0132', u'Drug Equipment Violations', u'371190055053013'), (u'0311', u'Weapon Law Violations', u'371190038033008'), (u'0425', u'All Other Offenses', u'371190055053013'), (u'0445', u'Motor Vehicle Theft', u'371190015043000'), (u'0626', u'Motor Vehicle Theft', u'371190019133000'), (u'0807', u'Theft From Motor Vehicle', u'371190012003029'), (u'0911', u'Theft of Motor Vehicle Parts from Vehicle', u'371190027003002'), (u'1008', u'False Pretenses/Swindle', u'371190061023039'), (u'1051', u'Statutory Rape', u'371190019103000'), (u'1140', u'Burglary/B&E', u'371190060043012'), (u'1153', u'Theft From Motor Vehicle', u'371190029043001'), (u'1210', u'Trespass Of Real Property', u'371190019093013'), (u'1248', u'All Other Thefts', u'371190058083001'), (u'1355', u'Robbery', u'371190034003005'), (u'1433', u'Theft From Motor Vehicle', u'371190012003027'), (u'1439', u'Robbery', u'371190060013032'), (u'1506', u'All Other Thefts', u'371190060013032'), (u

In [ ]:
encoder = preprocessing.LabelEncoder()
encoder.fit(list(set(df["NIBRSclass"])))

In [3]:
# only number and class
date_dict = dict()
for i, row in df.iterrows():
    date = row["Report_Dat"][:-5]
    t = (row["Report_Dat"][-4:], row["NIBRSclass"])
    date_dict[date] = date_dict.get(date, []) + [t]
    #print row["Report_Dat"][-4:]
    
date_dict = {k: sorted(date_dict[k], key=lambda x: x[0]) for k in date_dict}
sequences = [ [v[1] for v in date_dict[k]] for k in sorted(date_dict.keys())]

In [15]:
%%time
# seqs = [ ["a",2,3], [3,4,5], ["a",2,3], [2,3,4,5] ]
freq_seqs = seqmining.freq_seq_enum([t[1] for t in date_dict["01/01/2010"] if k[2][-4] == "3"], 10)
print len(freq_seqs)
sorted(freq_seqs)

0
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 901 µs
